In [1]:
import numpy as np
import pandas as pd 
import sklearn.svm
import sklearn.model_selection
import sklearn.preprocessing
from sklearn.metrics import accuracy_score

In [34]:
data = pd.read_csv('jannis.csv')
# print(data.head())

X = np.array(data._get_numeric_data(),dtype=np.float32)[:,1:]
Y = np.array(data['class'],dtype=np.int32)
# # print(X,Y)
# # enc = sklearn.preprocessing.LabelEncoder()
# # Y = enc.fit_transform(Y)
# # print(enc.classes_)
# # print(enc.transform(enc.classes_))
# # print(Y) 
# # print(enc.classes_)
# # print(enc.transform(enc.classes_))

# from sklearn.datasets import load_iris
# iris = load_iris()
# X = iris.data
# Y = iris.target
X,Y = X[:5000],Y[:5000]
print("num classes = ",len(set(Y)))
print(X.shape,Y.shape)

num classes =  4
(5000, 54) (5000,)


In [30]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y, test_size=0.50, random_state=42)
# x_test, y_test = x_test[:1000], y_test[:1000]
print(x_train.shape, x_test.shape)
print(y_train)

(2500, 54) (2500, 54)
[3 1 2 ... 3 1 1]


In [31]:
from sklearn.svm import SVC
clf = SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
clf.fit(x_train, y_train) 

y_pred = clf.predict(x_test)
print(accuracy_score(y_test,y_pred,normalize=True))
print(y_pred)
print(y_test)

0.4716
[3 3 3 ... 3 3 3]
[2 3 2 ... 2 3 3]


In [32]:
import numpy as np
from sklearn import svm

def distances(data1,order=2.0):
    dist = np.sum( np.abs(data1)**order, axis=1 )**(1/order)
    return dist

def gaussianKernel(x1, x2, sigma=0.1):
    # Ensure that x1 and x2 are column vectors
    x1 = x1.flatten()
    x2 = x2.flatten()
    sim = np.exp(-np.sum(np.power((x1-x2),2))/float((sigma**2)))
    return sim

def gaussianKernelGramMatrixFull(X1, X2, sigma=0.1):   
    """(Pre)calculates Gram Matrix K"""
    gram_matrix = np.zeros((X1.shape[0], X2.shape[0]))
    for i, x1 in enumerate(X1):
        for j, x2 in enumerate(X2):
#             print(x1)
            x1 = x1.flatten()
            x2 = x2.flatten()
#             print(x1.shape,x2.shape)
            gram_matrix[i,j] = gaussianKernel(x1, x2,sigma=sigma1)
    return gram_matrix

actual_distances = distances(x_train-np.mean(x_train,axis=0),order=2.0)
d_50 = np.percentile(actual_distances,50)
sigma1 = d_50/((-np.log(0.5))**0.5)
def my_kernel(X,Y):
    return gaussianKernelGramMatrixFull(X,Y,sigma=sigma1)

clf = svm.SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', gamma='auto', kernel=my_kernel,
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
clf.fit(x_train, y_train)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto',
  kernel=<function my_kernel at 0x7f9e2df07840>, max_iter=-1,
  probability=False, random_state=None, shrinking=True, tol=0.001,
  verbose=False)

In [33]:
y_pred = clf.predict(x_test)
print(accuracy_score(y_test,y_pred,normalize=True))
print(y_pred)
print(y_test)

0.5432
[2 3 1 ... 2 3 3]
[2 3 2 ... 2 3 3]


In [39]:
import numpy as np
from sklearn import svm

def distances(data1,order=2.0):
    dist = np.sum( np.abs(data1)**order, axis=1 )**(1/order)
    return dist

def pgaussianKernel(x1, x2, p,sigma=0.1):
    # Ensure that x1 and x2 are column vectors
    x1 = x1.flatten()
    x2 = x2.flatten()
    sim = np.exp(-np.sum(np.power(np.abs(x1-x2),p))/float((sigma**p)))
    return sim

def pgaussianKernelGramMatrixFull(X1, X2, sigma=0.1):           
    """(Pre)calculates Gram Matrix K"""
    gram_matrix = np.zeros((X1.shape[0], X2.shape[0]))
    for i, x1 in enumerate(X1):
        for j, x2 in enumerate(X2):
#             print(x1)
            x1 = x1.flatten()
            x2 = x2.flatten()
#             print(x1.shape,x2.shape)
            gram_matrix[i,j] = pgaussianKernel(x1, x2,p,sigma=sigma1)
    return gram_matrix

for fr in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    print("fraction fr=",fr)
    actual_distances = distances(x_train-np.mean(x_train,axis=0),order=fr)
    d_5, d_50, d_95 = np.percentile(actual_distances,5),np.percentile(actual_distances,50),np.percentile(actual_distances,95)
    p = np.log(np.log(0.05)/np.log(0.95)) / np.log(d_95/d_5) 
    print("p=",p)
    sigma1 = d_50/((-np.log(0.50))**(1/p))
    print("sigma1=",sigma1)
    def my_kernel(X,Y):
        return pgaussianKernelGramMatrixFull(X,Y,sigma=sigma1)

    clf = svm.SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
        decision_function_shape='ovr', gamma='auto', kernel=my_kernel,
        max_iter=-1, probability=False, random_state=None, shrinking=True,
        tol=0.001, verbose=False)
    clf.fit(x_train, y_train)

    y_pred = clf.predict(x_test)
    print(accuracy_score(y_test,y_pred,normalize=True))
#     print(y_pred)
#     print(y_test)

fraction fr= 0.1
p= 6.019041475771402
sigma1= 3.850526511040044e+17
0.4716
fraction fr= 0.2
p= 5.816376252216554
sigma1= 1093970561.182266
0.4716
fraction fr= 0.3
p= 5.584662114059132
sigma1= 1814839.8344001162
0.4716
fraction fr= 0.4
p= 5.462982277815125
sigma1= 81935.03139623748
0.4716
fraction fr= 0.5
p= 5.332001710922396
sigma1= 13712.88590661967
0.4716
fraction fr= 0.6
p= 5.242811862967891
sigma1= 4374.549668585338
0.4716
fraction fr= 0.7
p= 5.159172690425566
sigma1= 2001.6975841593655
0.4716
fraction fr= 0.8
p= 5.109798782229108
sigma1= 1140.5167164920053
0.4344
fraction fr= 0.9
p= 5.068694660725522
sigma1= 748.3461865901545
0.374


In [37]:
y_pred = clf.predict(x_test)
print(accuracy_score(y_test,y_pred,normalize=True))
print(y_pred)
print(y_test)

0.4716
[3 3 3 ... 3 3 3]
[2 3 2 ... 2 3 3]
